План:
1. зашифровать инт и инф?
2. метаданные предложений (спикеры)
3. лемматизация слов
3. добавить шифры интервью в мета
4. добавить аудио файлом, а не ссылкой

**Document-level meta:** filename, title, author, recplace, year, month, day, informants_id, is_transcript, audio_links

**Sentence-level meta:** speaker


In [ ]:
! pip install python-docx
! pip install dropbox

import os
import docx
import re
import json
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string
import itertools

import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
import urllib

from google.colab import files
from datetime import date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=d4b4d560f5b41822549bf3916e5577065373af0c32567c3aff295b0da7caf83f
  Stored in directory: /root/.cache/pip/wheels/83/8b/7c/09ae60c42c7ba4ed2dddaf2b8b9186cb105255856d6ed3dba5
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Парсинг расшифровки в json**

In [ ]:
def parse(path):
  #отрывки
  reg_line = re.compile("Ин..?:.*")
  reg_speaker = re.compile("(Ин..?):.*")
  reg_saying = re.compile("Ин..?:\s?(.*)") 

  #достаём текст
  doc = docx.Document(path)
  text = []
  for paragraph in doc.paragraphs:
      text.append(paragraph.text)
  text = '\n'.join(text)
  speakers_di = {}

  sentences_list = []
  lines = re.findall(reg_line, text)
  #utterance_number = 1
  for line in lines:
    sentence_di = {}
    speaker = re.findall(reg_speaker, line)[0]
    saying = re.findall(reg_saying, line)[0]
    sentence_di["lang"] = 0
    sentence_di["speaker"] = speaker
    sentence_di["text"] = saying
    #sentence_di["utterance_index"] = utterance_number
    sentences_list.append(sentence_di)
    #utterance_number += 1
    words = []
    tokens = word_tokenize(saying)
    number_of_tokens = len(tokens)
    index = 0
    index_neg = number_of_tokens
    off_start = 0
    for word in tokens:
      word_di = {}
      word_di["wf"] = word
      if word in string.punctuation:
        word_di["wtype"] = "punct"
      else:
        word_di["wtype"] = "word"
        word_di["sentence_index"] = index
        index += 1
        word_di["sentence_index_neg"] = index_neg
        index_neg -= 1
      word_di["off_start"] = off_start
      off_end = off_start + len(word)
      word_di["off_end"] = off_end
      off_start = off_end
      if word in string.punctuation:
        off_start = off_start + 1
      index += 1
      word_di["next_word"] = index
      words.append(word_di)
    sentence_di["words"] = words

  return sentences_list

In [ ]:
sentences_list = parse("int1.docx")
sentences_list

In [ ]:
meta_di = {}
meta_di["filename"] = "int1.docx"
meta_di["title"] = "5"
meta_di["author"] = "DAM, PAN"
meta_di['recplace'] = "Калевала"
meta_di["year"] = 2021

In [ ]:
json_di = {"meta" : meta_di, "sentences" : sentences_list}
with open("example_words.json", "w", encoding='utf-8') as jsonfile:
        json.dump(json_di, jsonfile, ensure_ascii=False)

In [ ]:
    words = []
    tokens = word_tokenize("[нрзб] tačʼe taos.")
    number_of_tokens = len(tokens)
    index = 0
    index_neg = number_of_tokens
    off_start = 0
    for word in tokens:
      word_di = {}
      word_di["wf"] = word
      if word in string.punctuation:
        word_di["wtype"] = "punct"
      else:
        word_di["wtype"] = "word"
        word_di["sentence_index"] = index
        index += 1
        word_di["sentence_index_neg"] = index_neg
        index_neg -= 1
      word_di["off_start"] = off_start
      off_end = off_start + len(word)
      word_di["off_end"] = off_end
      off_start = off_end
      if word in string.punctuation:
        off_start = off_start + 1
      index += 1
      word_di["next_word"] = index
      words.append(word_di)
    print(words)

[{'wf': '[', 'wtype': 'punct', 'off_start': 0, 'off_end': 1, 'next_word': 1}, {'wf': 'нрзб', 'wtype': 'word', 'sentence_index': 1, 'sentence_index_neg': 6, 'off_start': 2, 'off_end': 6, 'next_word': 3}, {'wf': ']', 'wtype': 'punct', 'off_start': 6, 'off_end': 7, 'next_word': 4}, {'wf': 'tačʼe', 'wtype': 'word', 'sentence_index': 4, 'sentence_index_neg': 5, 'off_start': 8, 'off_end': 13, 'next_word': 6}, {'wf': 'taos', 'wtype': 'word', 'sentence_index': 6, 'sentence_index_neg': 4, 'off_start': 13, 'off_end': 17, 'next_word': 8}, {'wf': '.', 'wtype': 'punct', 'off_start': 17, 'off_end': 18, 'next_word': 9}]


In [ ]:
word_tokenize("[нрзб] tačʼe taos...")

['[', 'нрзб', ']', 'tačʼe', 'taos', '...']

Старое

In [ ]:
#метаинформация интервью
reg_number = re.compile("Номер интервью в базе.*(.[\d])")
reg_place = re.compile("Место\:\s?(.*)")
reg_data = re.compile("Дата\:\s?(.[\d.]*)")
reg_inf_names = re.compile("_?.*_(.*)_.*\.docx")
reg_int = re.compile("Инт.*—\s?(.*)")

#информант и интервьюер
reg_inf = re.compile("Инф.*—\s?(.*)")
reg_gender = re.compile("(\w{1})\,")
reg_birth_year = re.compile("\d{4}")
reg_int = re.compile("Инт.*—\s?(.*)")
reg_role_int = re.compile("(Инт.*)\s—\s?.*")
reg_role_inf = re.compile("(Инф.*)\s—\s?.*")

#отрывки
reg_line = re.compile("Ин..?:.*")
reg_speaker = re.compile("(Ин..?):.*")
reg_saying = re.compile("Ин..?:\s?(.*)")

#словарь интервьюеров и шифров
intdict = {'AAA':'Анна Аксёнова', 'AAE':'Анна Елагина', 'AAS':'Анастасия Смирнова', 'AAT':'Анастасия Тимошина', 'ADI':'Анастасия  Иванова', 'ADU':'Алёна Утробина', 'AMR':'Алексей  Рыжков', 'ARZ':'Алина Заболотская', 'AVV':'Андриан Влахов', 'AXT':'Алина Тиллабаева', 'DAM':'Дарья Морозова'}

In [ ]:
path = '2022_03_04_Картаёль_Василий_Александрович_Кожевин_Иванова,_Маринина.docx'

In [ ]:
path = "int1.docx"

In [ ]:
meta_di = {}
meta_di["filename"] = path    

#достаём текст
doc = docx.Document(path)
text = []
for paragraph in doc.paragraphs:
    text.append(paragraph.text)
text = '\n'.join(text)
speakers_di = {}
    
number_in_db = re.findall(reg_number, text)[0]
place = re.findall(reg_place, text)[0]
meta_di["recplace"] = place

meta_di["title"] = number_in_db

#парсер даты
data = re.findall(reg_data, text)[0]
day = re.findall(r"(.*)\..*\..*", data)[0]
month = re.findall(r".*\.(.*)\..*", data)[0]
year = re.findall(r".*\..*\.(.*)", data)[0]
data = year+month+day
meta_di["year"] = data

meta_di["genre"] = "interview"

interviewers = re.findall(reg_int, text)
new_interviewers = interviewers
#new_interviewers = []
#for interviewer in interviewers:
 # if interviewer in 
  #new_interviewers.append(intdict[interviewer])
meta_di["author"] = new_interviewers


sentences_list = []
lines = re.findall(reg_line, text)
#utterance_number = 1
for line in lines:
    sentence_di = {}
    speaker = re.findall(reg_speaker, line)[0]
    saying = re.findall(reg_saying, line)[0]
    sentence_di["speaker"] = speaker
    sentence_di["text"] = saying
    sentence_di["lang"] = 1
    #sentence_di["utterance_index"] = utterance_number
    sentences_list.append(sentence_di)
    #utterance_number += 1
    words = saying.split()
    sentence_di["words"] = words

In [ ]:
meta_di

{'filename': 'int1.docx',
 'recplace': 'посёлок городского типа Калевала',
 'title': ' 5',
 'year': '202122.1',
 'genre': 'interview',
 'author': ['Дарья Морозова', 'Полина Недайхлеб']}

In [ ]:
meta_di = {}
meta_di["filename"] = "int1.docx"
meta_di["title"] = "5"
meta_di["author"] = "DAM, PAN"
meta_di['recplace'] = "Калевала"
meta_di["year"] = 2021


In [ ]:
json_di = {"meta" : meta_di, "sentences" : sentences_list}

Запись в json

In [ ]:
with open("example_new.json", "w", encoding='utf-8') as jsonfile:
        json.dump(json_di, jsonfile, ensure_ascii=False)

**Парсинг таблицы**

In [ ]:
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError

In [ ]:
url = 'https://www.dropbox.com/sh/1d1stbh1t6wm4nw/AAAtFaXlLP0lglLRxcCFBgJ5a/%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D0%B5%20%D1%84%D0%B0%D0%B9%D0%BB%D1%8B%20%D0%BF%D0%BE%20%D0%B8%D1%82%D0%BE%D0%B3%D0%B0%D0%BC%20%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D1%8C%D1%8E/%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D1%8C%D1%8E%2025%20%D0%B8%D1%8E%D0%BD%D1%8F/%D0%92%D0%B0%D1%81%D0%B8%D0%BB%D0%B8%D1%81%D0%B0%20%D0%94%D1%91%D0%BC%D0%B8%D0%BD%D0%B0%20-%20%D0%9D%D0%B8%D0%BA%D0%B8%D1%82%D0%B0%20%D0%A1%D1%82%D0%B5%D0%BF%D0%B0%D0%BD%D0%BE%D0%B2.docx?dl=0'

**Достаём данные из таблицы**

! file = ИМЯ БАЗЫ В ЭКСЕЛЕ

In [ ]:
file = 'the_base.xlsx'

base = pd.ExcelFile(file)
print(base.sheet_names)

['интервью', 'информанты', 'интервьюеры', 'экспедиции', 'населённые пункты']


In [ ]:
base_interviews = base.parse('интервью')
base_interviews

,ID интервью,Код экспедиции,Шифр для цитат (1-CUM22),Текущие названия файлов,Длины файлов,Нужна правка аудио,Формат записи,Ссылка на аудио,Дата записи интервью,Место записи интервью,...,Ссылка на расшифровку,ИНФ1 (основной),ИНФ2 (основной),ИНФ3 (основной),ИНФ4,ИНФ5,ИНФ6,ИНФ7,Тэги,Комментарии («помойка»)
0,1.0,KAI19,1-KAI19-1,"1-KAI19-1.MP3, Полина Александрова - 25.06, Та...",00:55:11,NaN,mp3,https://www.dropbox.com/sh/vqcvd8wuyinj4cm/AAD...,2019-06-25 00:00:00,Петрозаводск,...,http://lingconlab.ru/karelrus/files_html/20190...,i3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,KAI19,2-KAI19-1,2-KAI19-1.m4a,00:15:28,NaN,m4a,https://www.dropbox.com/sh/vqcvd8wuyinj4cm/AAD...,2019-06-24 00:00:00,Петрозаводск,...,NaN,i143,i144,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,KAI19,3-KAI19-1,"3-KAI19-1.m4a, В - 25.06 Игорь.docx",00:04:38,NaN,m4a,https://www.dropbox.com/sh/vqcvd8wuyinj4cm/AAA...,2019-06-25 00:00:00,Петрозаводск,...,https://www.dropbox.com/sh/1d1stbh1t6wm4nw/AAD...,i35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,KAI19,3-KAI19-2,3-KAI19-2.m4a,00:19:21,NaN,m4a,https://www.dropbox.com/sh/vqcvd8wuyinj4cm/AAD...,2019-06-25 00:00:00,Петрозаводск,...,https://www.dropbox.com/sh/1d1stbh1t6wm4nw/AAD...,i35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,KAI19,4-KAI19-1,"4-KAI19-1.m4a, Ekaterina Shcheglova - Алёна_Ви...",00:08:57,NaN,m4a,https://www.dropbox.com/sh/vqcvd8wuyinj4cm/AAD...,2019-06-25 00:00:00,Петрозаводск,...,https://www.dropbox.com/sh/1d1stbh1t6wm4nw/AAB...,i145,i146,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,689.0,KOM22,689-KOM22-1,"11_Гам_Бабикова Инна Николаевна, …евна Чупрова...",01:04:26,NaN,wav,NaN,2022-03-11 00:00:00,Гам,...,NaN,i917,i918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
810,690.0,KOM22,690-KOM22-1,2022-03-11_Гам_Нина Николаевна Чупрова_Широков...,02:14:13,NaN,wav,NaN,2022-03-11 00:00:00,Гам,...,NaN,i918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811,691.0,KOM22,691-KOM22-1,"11_Ласта_Егор Николаевич Канев, Т…вна Ануфриев...",04:08:30,NaN,mp3,NaN,2022-03-11 00:00:00,Ласта,...,NaN,i919,i920,NaN,NaN,NaN,NaN,NaN,NaN,NaN
812,692.0,KOM22,692-KOM22-1,11_Ижма_Ирина Королёва_Влахов.MP3\n,01:11:02,NaN,mp3,NaN,2022-03-11 00:00:00,Ижма,...,NaN,i921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(base_interviews)

814

In [ ]:
base_interviews['Текущие названия файлов'][0]

'1-KAI19-1.MP3, Полина Александрова - 25.06, Татьяна, Полина А + Андриан.docx'

In [ ]:
base_interviews['Ссылка на расшифровку'][0]

'http://lingconlab.ru/karelrus/files_html/20190625%D0%B0_Petrozavodsk_TB1961.html'

In [ ]:
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
from dropbox import DropboxOAuth2FlowNoRedirect

Подключение в DropBoxAPI

In [ ]:
APP_KEY = '55pknt48n0zltjr'
APP_SECRET = 'q8efrqad1q0rnns'
auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)
authorize_url = auth_flow.start()
print ("1. Go to: " + authorize_url)
print ("2. Click \"Allow\" (you might have to log in first).")
print ("3. Copy the authorization code.")
auth_code = input("Enter the authorization code here: ").strip()
try:
 oauth_result = auth_flow.finish(auth_code)
except Exception as e:
  print("Error: %s" % (e,))

1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=55pknt48n0zltjr
2. Click "Allow" (you might have to log in first).
3. Copy the authorization code.
Enter the authorization code here: rXidN-vZ2ZAAAAAAAAAAI7ELtDSwji8ZXzbavqWR6kw


In [ ]:
dbx = dropbox.Dropbox("sl.BdQxhmCYO85Esp0JriQMJjehapB401lb_S9N2y7bw5HwtlmuFV-BNDZrKKra5K_ctZ7p6rzEugPqOk6mNv-Qcs7HXq7yqePNb3aeOKCSY1Yrh_H-QBqJMPoZKGa87C6pZP1343RX")

**Пример работы со ссылкой:**

Норм ссылка:

In [ ]:
url = "https://www.dropbox.com/sh/ggznkk19jn7k4gm/AAAz2nWm3Uy9G6EaUZdKhnoNa/Anastasia%20Smirnova%20-%202021-11-22_%D0%9A%D0%B0%D0%BB%D0%B5%D0%B2%D0%B0%D0%BB%D0%B0_%D0%93%D0%B0%D0%BB%D0%B8%D0%BD%D0%B0%20%D0%92%D0%B8%D0%BB%D1%85%D0%BE%D0%B2%D0%BD%D0%B0_%D0%9C%D0%BE%D1%80%D0%BE%D0%B7%D0%BE%D0%B2%D0%B0%2C%20%D0%9D%D0%B5%D0%B4%D0%B0%D0%B9%D1%85%D0%BB%D0%B5%D0%B1.docx?dl=0"

In [ ]:
yrl = 'https://www.dropbox.com/sh/r4vbikfvm7a8iqk/AADVpvkwUJK7Uj5SiYOxNdE4a/2021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9%20%D0%A1%D0%B5%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2_%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0%2C%20%D0%9B%D0%BE%D0%B1%D0%B0%D0%BD%D0%BE%D0%B2%D0%B0.docx?dl=0'

НЕ норм ссылка:

Ссылка должна быть без букв кириллицы, поэтому мы её перекодируем

In [ ]:
url = "https://www.dropbox.com/sh/ggznkk19jn7k4gm/AAAz2nWm3Uy9G6EaUZdKhnoNa/Anastasia%20Smirnova%20-%202021-11-22_Калевала_Галина%20Вилховна_Морозова%2C%20Недайхлеб.docx?dl=0"

In [ ]:
reg_bad_url = re.compile("https:\/\/www\.dropbox\.com\/sh\/.*\/(.*)\.docx")
reg_right_url_part = re.compile("(https:\/\/www\.dropbox\.com\/sh\/.*\/).*\.docx")

In [ ]:
right_url_part = re.findall(reg_right_url_part, url)[0]
print(right_url_part)

https://www.dropbox.com/sh/r4vbikfvm7a8iqk/AADVpvkwUJK7Uj5SiYOxNdE4a/


In [ ]:
bad_name = re.findall(reg_bad_url, url)[0]
bad_name = bad_name.replace("%20", " ")
bad_name = bad_name.replace("%2C", ",")
print(bad_name)

2021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9 %D0%A1%D0%B5%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2_%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0, %D0%9B%D0%BE%D0%B1%D0%B0%D0%BD%D0%BE%D0%B2%D0%B0


In [ ]:
right_name = urllib.parse.quote(bad_name)
print(right_name)

2021-07-26_%25D0%2598%25D0%25B6%25D0%25BC%25D0%25B0_%25D0%2590%25D0%25BB%25D0%25B5%25D0%25BA%25D1%2581%25D0%25B5%25D0%25B9%20%25D0%25A1%25D0%25B5%25D0%25BC%25D0%25B5%25D0%25BD%25D0%25BE%25D0%25B2_%25D0%2592%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B4%25D0%25B8%25D0%25BD%25D0%25B0%2C%20%25D0%259B%25D0%25BE%25D0%25B1%25D0%25B0%25D0%25BD%25D0%25BE%25D0%25B2%25D0%25B0


In [ ]:
right_url = right_url_part + right_name + ".docx?dl=0"
print(right_url)

https://www.dropbox.com/sh/r4vbikfvm7a8iqk/AADVpvkwUJK7Uj5SiYOxNdE4a/2021-07-26_%25D0%2598%25D0%25B6%25D0%25BC%25D0%25B0_%25D0%2590%25D0%25BB%25D0%25B5%25D0%25BA%25D1%2581%25D0%25B5%25D0%25B9%20%25D0%25A1%25D0%25B5%25D0%25BC%25D0%25B5%25D0%25BD%25D0%25BE%25D0%25B2_%25D0%2592%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B4%25D0%25B8%25D0%25BD%25D0%25B0%2C%20%25D0%259B%25D0%25BE%25D0%25B1%25D0%25B0%25D0%25BD%25D0%25BE%25D0%25B2%25D0%25B0.docx?dl=0


Обращение к дропбоксу

In [ ]:
md, res = dbx.sharing_get_shared_link_file(url=right_url)

In [ ]:
res

<Response [200]>

In [ ]:
type(res)

requests.models.Response

значит, всё ок

md = метаданные, res - содержание

Запишем содержание в ворд, чтобы распарсить

In [ ]:
with open("int1.docx", "wb") as f:
    f.write(res.content)

**Document-level meta:** filename, title, interviewers, place, year, month, day, informants, is_transcript, audio_links

**Sentence-level meta:** speaker


**Функция парсинга:**

Возвращает список предложений

**Задачи:**

*   дата в мета интервью 
*   леммы добавить

In [ ]:
! pip install natasha

from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=f74c1647768e951598d87a40fde38c29e8cd5c2ea28c762671b82b88c619687d
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26114 sha256=44f8f11d30d3fffe4f6a703cedaf32d9cc4757489f92beca6fb93d5a9795cb18
  Stored in directory: /root/.cache/pip/wheels/ab/fa/1b/75d9a71327979678

**Правильная функция:**

In [ ]:
def parse(path, infs, ints):
  #отрывки
  reg_line = re.compile("Ин..?:.*")
  reg_speaker = re.compile("(Ин..?):.*")
  reg_number = re.compile("\d")
  reg_saying = re.compile("Ин..?:\s?(.*)") 

  #достаём текст
  doc = docx.Document(path)
  text = []
  for paragraph in doc.paragraphs:
      text.append(paragraph.text)
  text = '\n'.join(text)
  speakers_di = {}

  sentences_list = []
  lines = re.findall(reg_line, text)
  for line in lines:
    sentence_di = {}
    sentence_di["lang"] = 0
    saying = re.findall(reg_saying, line)[0]
    speaker = re.findall(reg_speaker, line)[0]
    if 'Инт' in speaker:
      if len(re.findall(reg_number, speaker)) > 0:
        int_number = int(re.findall(reg_number, speaker)[0])
      else:
        int_number = 1
      int_number -= 1
      try:
        speaker = ints[int_number]
      except IndexError:
        speaker = 'Somebody'
    elif 'Инф' in speaker:
      if len(re.findall(reg_number, speaker)) > 0:
        inf_number = int(re.findall(reg_number, speaker)[0])
      else:
        inf_number = 1
      inf_number -= 1
      try:
        speaker = infs[inf_number]
      except IndexError:
        speaker = 'Somebody'
    else:
      speaker = 'Somebody'

    sentence_meta_di = {}
    sentence_meta_di["speaker"] = speaker
    sentence_di["text"] = saying
    sentences_list.append(sentence_di)

    words = []
    li = [[word_tokenize(w), ' '] for w in saying.split()]
    tokens = list(itertools.chain(*list(itertools.chain(*li))))
    index = 0
    off_start = 0
    prev_space = 0
    word_counter = 0

    morph = morph_analyse(saying)

    for word in tokens:
      word_di = {}
      word_di["wf"] = word
      if word in string.punctuation:
        word_di["wtype"] = "punct"
      elif word == ' ':
        prev_space = 1
      else:
        word_di["wtype"] = "word"
        if word in morph.keys():
          word_di["ana"] = [morph[word]]
        word_counter += 1
        
      if prev_space == 1:
        off_start = off_start + 1
        prev_space = 0
      
      if word != ' ':
        word_di["off_start"] = off_start
        off_end = off_start + len(word)
        word_di["off_end"] = off_end
        off_start = off_end
        index += 1
        word_di["next_word"] = index
        
      if word != ' ':
        words.append(word_di)
    
    snt_indx = 0
    for word in words:
      if word['wtype'] == 'word':
        word["sentence_index_neg"] = word_counter
        word_counter -= 1
      if word['wtype'] != 'punct':
        word["sentence_index"] = snt_indx
        snt_indx += 1
      elif word['wtype'] == 'punct' and word['off_start'] != 0 and word['next_word'] < len(words):
        word["sentence_index"] = snt_indx
        snt_indx += 1
    
    sentence_di["words"] = words
    sentence_di["meta"] = sentence_meta_di

  return sentences_list

In [ ]:
def morph_analyse(sentence):
  ana_list = {}
  seq = sentence
  doc = Doc(seq)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  ana_di = {}
  
  for token in doc.tokens:
    token.lemmatize(morph_vocab)

  for token in doc.tokens:
    if token.pos != 'PUNCT' and token.pos != 'X':
      ana_di["lex"] = token.lemma
      ana_di["gr.pos"] = token.pos
      if 'Animacy' in token.feats.keys():
        ana_di["gr.anim"] = token.feats['Animacy']
      if 'Case' in token.feats.keys():
        ana_di["gr.case"] = token.feats['Case']
      if 'Number' in token.feats.keys():
        ana_di["gr.number"] = token.feats['Number']
      if 'Person' in token.feats.keys():
        ana_di["gr.person"] = token.feats['Person']
      if 'Aspect' in token.feats.keys():
        ana_di["gr.aspect"] = token.feats['Aspect']
      if 'Mood' in token.feats.keys():
        ana_di["gr.mood"] = token.feats['Mood']
      if 'Tense' in token.feats.keys():
        ana_di["gr.tense"] = token.feats['Tense']
      if 'VerbForm' in token.feats.keys():
        ana_di["gr.verbform"] = token.feats['VerbForm']
      if 'Voice' in token.feats.keys():
        ana_di["gr.voice"] = token.feats['Voice']
      if 'Degree' in token.feats.keys():
        ana_di["gr.degree"] = token.feats['Degree']
      ana_list[token.text] = ana_di
      ana_di = {}

  return ana_list

In [ ]:
sent = 'А вы, получается, вы здесь родились? родившись в 2022'
morph_analyse(sent)

{'А': {'lex': 'а', 'gr.pos': 'CCONJ'},
 'вы': {'lex': 'вы',
  'gr.pos': 'PRON',
  'gr.case': 'Nom',
  'gr.number': 'Plur',
  'gr.person': '2'},
 'получается': {'lex': 'получаться',
  'gr.pos': 'VERB',
  'gr.number': 'Sing',
  'gr.person': '3',
  'gr.aspect': 'Imp',
  'gr.mood': 'Ind',
  'gr.tense': 'Pres',
  'gr.verbform': 'Fin',
  'gr.voice': 'Mid'},
 'здесь': {'lex': 'здесь', 'gr.pos': 'ADV', 'gr.degree': 'Pos'},
 'родились': {'lex': 'родиться',
  'gr.pos': 'VERB',
  'gr.number': 'Plur',
  'gr.aspect': 'Perf',
  'gr.mood': 'Ind',
  'gr.tense': 'Past',
  'gr.verbform': 'Fin',
  'gr.voice': 'Mid'},
 'родившись': {'lex': 'родиться',
  'gr.pos': 'VERB',
  'gr.number': 'Plur',
  'gr.aspect': 'Perf',
  'gr.mood': 'Ind',
  'gr.tense': 'Past',
  'gr.verbform': 'Fin',
  'gr.voice': 'Mid'},
 'в': {'lex': 'в', 'gr.pos': 'ADP'},
 '2022': {'lex': '2022', 'gr.pos': 'ADJ'}}

In [ ]:
sentences_list = parse('int1.docx', ['i1'], ['DM', 'PN'])
meta_di = {}
meta_di["filename"] = "int1.docx"
meta_di["title"] = "5"
meta_di["author"] = "DAM, PAN"
meta_di['recplace'] = "Калевала"
meta_di["year"] = 2021

json_di = {"meta" : meta_di, "sentences" : sentences_list}
with open("example_words.json", "w", encoding='utf-8') as jsonfile:
        json.dump(json_di, jsonfile, ensure_ascii=False)

In [ ]:
sent = 'А вы, получается, вы здесь родились?'
li = [[word_tokenize(w), ' '] for w in sent.split()]
tokens = list(itertools.chain(*list(itertools.chain(*li))))

In [ ]:
tokens

['А',
 ' ',
 'вы',
 ',',
 ' ',
 'получается',
 ',',
 ' ',
 'вы',
 ' ',
 'здесь',
 ' ',
 'родились',
 '?',
 ' ']

In [ ]:
for i in range(len(base_interviews)):
  recording_data = base_interviews['Дата записи интервью'][i]
  print(str(i), recording_data, type(recording_data))
  if type(recording_data) is float:
    print('yes')

**Код, который проходится по всем интервью из таблицы, записывает их метаданные (из таблицы) и парсит расшифровку (если она есть). Всё записывается в json и скачивается на компьютер. 1 json = 1 интервью**

In [ ]:
re.compile("Инт.*—\s?(.*)")

In [ ]:
cyrillic = re.compile("А-яа-я]")

In [ ]:
cyrillic

re.compile(r'А-яа-я]', re.UNICODE)

In [ ]:
from dropbox.exceptions import ApiError

In [ ]:
cyrillic = re.compile("А-яа-я]")
reg_bad_url = re.compile("https:\/\/www\.dropbox\.com\/sh\/.*\/(.*)\.docx")
reg_right_url_part = re.compile("(https:\/\/www\.dropbox\.com\/sh\/.*\/).*\.docx")

for i in range(len(base_interviews)):
  file_di = {}
  all_files = base_interviews['Текущие названия файлов'][i]
  expedition = base_interviews['Код экспедиции'][i]
  interview_citation = base_interviews['Шифр для цитат (1-CUM22)'][i]
  if type(base_interviews['Ссылка на аудио'][i]) == str:
    audio_links = base_interviews['Ссылка на аудио'][i].split(',')
  else:
    audio_links = ['none']
  recording_data = base_interviews['Дата записи интервью'][i]
  if type(recording_data) is str:
    if ' и ' in recording_data:
      recording_data = recording_data.split(' и ')[0]
      year, month, day = recording_data.split('.')
    elif ', ' in recording_data:
      recording_data = recording_data.split(', ')[0]
      year, month, day = recording_data.split('.')
    else:
      year, month, day = 0, 0, 0
  elif type(recording_data) is not float:
    year = int(recording_data.strftime("%Y"))
    month = int(recording_data.strftime("%m"))
    day = int(recording_data.strftime("%d"))
  else:
    year = 0
    month = 0
    day = 0
  recording_place = base_interviews['Место записи интервью'][i]
  if type(base_interviews['Интервьюеры'][i]) == str:
    interviewers = base_interviews['Интервьюеры'][i].split(',')
  else:
    interviewers = ['Somebody']
  concern = base_interviews['Согласие на запись'][i]
  is_transcript = base_interviews['Расшифровка'][i]
  transcript_format = base_interviews['Формат расшифровки'][i]
  inf1 = base_interviews['ИНФ1 (основной)'][i]
  inf2 = base_interviews['ИНФ2 (основной)'][i]
  inf3 = base_interviews['ИНФ3 (основной)'][i]
  inf4 = base_interviews['ИНФ4'][i]
  inf5 = base_interviews['ИНФ5'][i]
  inf6 = base_interviews['ИНФ6'][i]
  inf7 = base_interviews['ИНФ7'][i]
  informants = [inf1, inf2, inf3, inf4, inf5, inf6, inf7]
  true_informants = []
  for inf in informants:
    if type(inf) == str:
      true_informants.append(inf)
  meta_di = {}
  meta_di["title"] = interview_citation
  meta_di["year"] = year
  meta_di["month"] = month
  meta_di["day"] = day
  meta_di["place"] = recording_place
  meta_di["interviewers"] = ", ".join(interviewers)
  meta_di["filename"] = interview_citation
  #filename не будет видно в корпусе
  meta_di["informants"] = ", ".join(true_informants)
  #if transcript_format != '':
   # is_transcript = is_transcript + ', ' + transcript_format
  meta_di["is_transcript"] = is_transcript
  meta_di["audio_links"] = audio_links
  file_di["meta"] = meta_di

  #парсим расшифровку, если она есть
  wrong_links = []
  if is_transcript == 'есть' and transcript_format == 'дословная':
    transcript_links = base_interviews['Ссылка на расшифровку'][i].split(',')
    for url in transcript_links:
      if "preview" not in url and "dropbox" in url:
        #and '.TextGrid' not in url
        if len(re.findall(cyrillic, url)) == 0:
          right_url = url
        else:
          right_url_part = re.findall(reg_right_url_part, url)[0]
          bad_name = re.findall(reg_bad_url, url)[0]
          bad_name = bad_name.replace("%20", " ")
          bad_name = bad_name.replace("%2C", ",")
          right_name = urllib.parse.quote(bad_name)
          right_url = right_url_part + right_name + ".docx?dl=0"
        try:
          md, res = dbx.sharing_get_shared_link_file(url=right_url)
        except ApiError:
          break
        with open("int_text.docx", "wb") as f:
          f.write(res.content)
          sentences_list = parse("int_text.docx", true_informants, interviewers)
        file_di["sentences"] = sentences_list
        print(file_di)
        break
      else:
        wrong_links.append(url)
  else:
    file_di["sentences"] = [] 
  with open("/content/jsons/inf" + str(i) + ".json", "w", encoding='utf-8') as jsonfile:
        json.dump(file_di, jsonfile, ensure_ascii=False)
  #files.download("/content/jsons/inf" + str(i) + ".json")

In [ ]:
re.findall(reg_bad_url, url)[0]

'2021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9%20%D0%A1%D0%B5%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2_%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0%2C%20%D0%9B%D0%BE%D0%B1%D0%B0%D0%BD%D0%BE%D0%B2%D0%B0'

In [ ]:
len(re.findall(cyrillic, url)) == 0

True

In [ ]:
url

'https://www.dropbox.com/sh/cbv5r691yegkuc9/AACiRyDXiG1CwYp5FoA_0PI5a/аудио%20интервью/Настя%20Буракова%20-%202021-07-26_Ижма_Лидия%20Терентьева%20и%20Ольга%20Артеева_Дёмина%2C%20Широков.docx?dl=0'

In [ ]:
bad_name

'2021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9 %D0%A1%D0%B5%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2_%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0, %D0%9B%D0%BE%D0%B1%D0%B0%D0%BD%D0%BE%D0%B2%D0%B0'

In [ ]:
right_url

'https://www.dropbox.com/sh/cbv5r691yegkuc9/AACiRyDXiG1CwYp5FoA_0PI5a/аудио%20интервью/Настя%20Буракова%20-%202021-07-26_Ижма_Лидия%20Терентьева%20и%20Ольга%20Артеева_Дёмина%2C%20Широков.docx?dl=0'

In [ ]:
right_url_part

'https://www.dropbox.com/sh/cbv5r691yegkuc9/AACiRyDXiG1CwYp5FoA_0PI5a/аудио%20интервью/'

In [ ]:
bad_name

'Настя Буракова - 2021-07-26_Ижма_Лидия Терентьева и Ольга Артеева_Дёмина, Широков'

In [ ]:
right_name

'%D0%9D%D0%B0%D1%81%D1%82%D1%8F%20%D0%91%D1%83%D1%80%D0%B0%D0%BA%D0%BE%D0%B2%D0%B0%20-%202021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%9B%D0%B8%D0%B4%D0%B8%D1%8F%20%D0%A2%D0%B5%D1%80%D0%B5%D0%BD%D1%82%D1%8C%D0%B5%D0%B2%D0%B0%20%D0%B8%20%D0%9E%D0%BB%D1%8C%D0%B3%D0%B0%20%D0%90%D1%80%D1%82%D0%B5%D0%B5%D0%B2%D0%B0_%D0%94%D1%91%D0%BC%D0%B8%D0%BD%D0%B0%2C%20%D0%A8%D0%B8%D1%80%D0%BE%D0%BA%D0%BE%D0%B2'

In [ ]:
right_url

'https://www.dropbox.com/sh/cbv5r691yegkuc9/AACiRyDXiG1CwYp5FoA_0PI5a/аудио%20интервью/%D0%9D%D0%B0%D1%81%D1%82%D1%8F%20%D0%91%D1%83%D1%80%D0%B0%D0%BA%D0%BE%D0%B2%D0%B0%20-%202021-07-26_%D0%98%D0%B6%D0%BC%D0%B0_%D0%9B%D0%B8%D0%B4%D0%B8%D1%8F%20%D0%A2%D0%B5%D1%80%D0%B5%D0%BD%D1%82%D1%8C%D0%B5%D0%B2%D0%B0%20%D0%B8%20%D0%9E%D0%BB%D1%8C%D0%B3%D0%B0%20%D0%90%D1%80%D1%82%D0%B5%D0%B5%D0%B2%D0%B0_%D0%94%D1%91%D0%BC%D0%B8%D0%BD%D0%B0%2C%20%D0%A8%D0%B8%D1%80%D0%BE%D0%BA%D0%BE%D0%B2.docx?dl=0'

In [ ]:
          right_url_part = re.findall(reg_right_url_part, url)[0]
          bad_name = re.findall(reg_bad_url, url)[0]
          bad_name = bad_name.replace("%20", " ")
          bad_name = bad_name.replace("%2C", ",")
          right_name = urllib.parse.quote(bad_name)
          right_url = right_url_part + right_name + ".docx?dl=0"

In [ ]:
base_interviews['Дата записи интервью'][405].split(' и ')

['26.07.2021', '30.07.2021']

In [ ]:
print(str(i))

420


In [ ]:
print(true_informants)

['i469', 'i470']


In [ ]:
interviewers

['Somebody']

Проблемные ссылки:

i = 405
https://www.dropbox.com/sh/cbv5r691yegkuc9/AACpPj3RHr95ZSEJaDxQfI7Xa/аудио%20интервью/2021-07-26_Ижма_Элида%20Сметанина_Володина%2C%20Лобанова.docx?dl=0
папка!!

368-KOI21-1 ссылка на корзину!!

In [ ]:
def parse(path, infs, ints):
  #отрывки
  reg_line = re.compile("Ин..?:.*")
  reg_speaker = re.compile("(Ин..?):.*")
  reg_number = re.compile("\d")
  reg_saying = re.compile("Ин..?:\s?(.*)") 

  #достаём текст
  doc = docx.Document(path)
  text = []
  for paragraph in doc.paragraphs:
      text.append(paragraph.text)
  text = '\n'.join(text)
  speakers_di = {}

  sentences_list = []
  lines = re.findall(reg_line, text)
  for line in lines:
    sentence_di = {}
    sentence_di["lang"] = 0
    saying = re.findall(reg_saying, line)[0]
    speaker = re.findall(reg_speaker, line)[0]
    if 'Инт' in speaker:
      int_number = int(re.findall(reg_number, speaker)[0])
      int_number -= 1
      try:
        speaker = ints[int_number]
      except IndexError:
        speaker = 'Somebody'
    elif 'Инф' in speaker:
      if len(re.findall(reg_number, speaker)) > 0:
        inf_number = int(re.findall(reg_number, speaker)[0])
      else:
        inf_number = 1
      inf_number -= 1
      try:
        speaker = infs[inf_number]
      except IndexError:
        speaker = 'Somebody'
    else:
      speaker = 'Somebody'

    sentence_meta_di = {}
    sentence_meta_di["speaker"] = speaker
    sentence_di["text"] = saying
    sentences_list.append(sentence_di)

    words = []
    li = [[word_tokenize(w), ' '] for w in saying.split()]
    tokens = list(itertools.chain(*list(itertools.chain(*li))))
    index = 0
    off_start = 0
    prev_space = 0
    word_counter = 0

    morph = morph_analyse(saying)

    for word in tokens:
      word_di = {}
      word_di["wf"] = word
      if word in string.punctuation:
        word_di["wtype"] = "punct"
      elif word == ' ':
        prev_space = 1
      else:
        word_di["wtype"] = "word"
        if word in morph.keys():
          word_di["ana"] = [morph[word]]
        word_counter += 1
        
      if prev_space == 1:
        off_start = off_start + 1
        prev_space = 0
      
      if word != ' ':
        word_di["off_start"] = off_start
        off_end = off_start + len(word)
        word_di["off_end"] = off_end
        off_start = off_end
        index += 1
        word_di["next_word"] = index
        
      if word != ' ':
        words.append(word_di)
    
    snt_indx = 0
    for word in words:
      if word['wtype'] == 'word':
        word["sentence_index_neg"] = word_counter
        word_counter -= 1
      if word['wtype'] != 'punct':
        word["sentence_index"] = snt_indx
        snt_indx += 1
      elif word['wtype'] == 'punct' and word['off_start'] != 0 and word['next_word'] < len(words):
        word["sentence_index"] = snt_indx
        snt_indx += 1
    
    sentence_di["words"] = words
    sentence_di["meta"] = sentence_meta_di

  return sentences_list

In [ ]:
parse("int_text.docx", true_informants, interviewers)

In [ ]:
1####убрать потом!
print("Неверные ссылки:", wrong_links)

In [ ]:
for i in range (0, 29):
  files.download("inf" + str(i) + ".json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>